In [1]:
import numpy as np 
import pandas as pd
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

In [25]:
# num_subj = 2

# num_games = [4]
def FunctionGetSubData(n_subj, n_game): 
    n = 1
    while n <= n_subj:
        if n < 10: 
            emotions = "../GAMEEMO/(S0{})/Preprocessed EEG Data/.csv format/S0{}G{}AllChannels.csv".format(n, n, n_game)
        else:
            emotions = "../GAMEEMO/(S{})/Preprocessed EEG Data/.csv format/S{}G{}AllChannels.csv".format(n, n, n_game)
        data = pd.read_csv(emotions)
        data = data.drop(columns=["Unnamed: 14"])
        data["subject_num"] = n
        
        if n == 1:
            emo_df = data.iloc[15300:30600]
        else:
            # third minute of each subject's data
            emo_df2 = data.iloc[15300:30600]
            emo_df = pd.concat([emo_df, emo_df2], axis=0)
        # emotions1= emotions1.reset_index(drop=True)
        n = n+1
    
    # emotions["pred_valence"] = 1
    # emotions["pred_arousal"] = 1
    return (emo_df)
    # print(emotions_funny.info())


In [35]:
def FunctionGetGameData(n_subj, n_games):
    for game in n_games:
        # # 1 = bored (0,0); 2 = calm (1,0); 3 = horror (0,1); 4 = funny (1,1)
        if game == 1:
            emo_bored = FunctionGetSubData(n_subj, game)
            emo_bored["pred_valence"] = 0
            emo_bored["pred_arousal"] = 0
        elif game == 2:
            emo_calm = FunctionGetSubData(n_subj, game)
            emo_calm["pred_valence"] = 1
            emo_calm["pred_arousal"] = 0
        elif game == 3:
            emo_horror = FunctionGetSubData(n_subj, game)
            emo_horror["pred_valence"] = 0
            emo_horror["pred_arousal"] = 1
        elif game == 4:
            emo_funny = FunctionGetSubData(n_subj, game)
            emo_funny["pred_valence"] = 1
            emo_funny["pred_arousal"] = 1
    emotions_all = pd.concat([emo_bored, emo_calm, emo_horror, emo_funny], axis=0)
    return emotions_all



In [37]:
emo_all = FunctionGetGameData(1, [1, 2, 3, 4])
print(emo_all)

# emo_bored = FunctionGetSubData(6, 1)
# print(emo_bored)

           AF3       AF4       F3         F4       F7       F8      FC5  \
15300  -9.3930   9.40330   7.3046   6.108800 -11.4582  22.6112   7.6865   
15301  -6.4192   7.15210  14.3379   3.934500   3.3293  13.4209  21.3253   
15302   6.2582   0.21479   6.4875   0.083213  17.2295   4.6619   5.9458   
15303  -8.2173 -13.89810  -8.0017 -14.021800   2.0957  -9.7179  -8.5109   
15304 -12.6653 -21.37930 -13.9085 -27.280500  -2.2481 -14.5578  -2.9388   
...        ...       ...      ...        ...      ...      ...      ...   
30595  -7.6501  -4.00400   8.4080  -9.002600  23.2527   9.9594  18.3640   
30596 -11.7706   6.11840  -1.9786   5.637600   7.7575   9.6031   7.8621   
30597 -12.9927   7.19750  -3.3060   7.347900  -6.8079  -5.0731  -6.7096   
30598 -10.7667   1.94500   7.9797   3.532700   7.7006   2.5823   7.7930   
30599  -5.0595  -7.08980   3.4033  -0.776850  15.5537   1.7043   6.6023   

            FC6       O1       O2       P7       P8        T7       T8  \
15300   0.34389  10.3142 

In [39]:
# Separate Target Variable and Predictor Variables
TargetVariable=['pred_valence', 'pred_arousal']
# Predictors=['AF3', 'AF4', 'F3', 'F4']
Predictors=['AF3', 'AF4', 'F3', 'F4', 'F7', 'F8' ,'FC5', 'FC6', 'O1', 'O2', 'P7', 'P8', 'T7', 'T8']
X=emo_all[Predictors].values
y=emo_all[TargetVariable].values

### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)

# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(42840, 14)
(42840, 2)
(18360, 14)
(18360, 2)


In [41]:
# importing the libraries
from tensorflow import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential, load_model

from keras.optimizers import SGD
from keras.models import Sequential
from sklearn.metrics import mean_absolute_percentage_error

In [42]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10]
    epoch_list  =   [10]
    
    import pandas as pd
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            # create ANN model
            model = Sequential()
            # Defining the first layer of the model
            model.add(Dense(units=10, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
 
            # applying dropout to layer
            model.add(Dropout(0.1))

            # Defining the Second layer of the model
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))

            # Defining the third layer of the model
            model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
 
            # applying dropout to layer
            model.add(Dropout(0.1))

            # The output neuron is a single fully connected node 
            # Since we will be predicting a single number
            model.add(Dense(2, kernel_initializer='normal'))
 
            # Compiling the model
            model.compile(loss='mean_squared_error', optimizer='adam')
 
            # Fitting the ANN to the Training set
            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=2)
 
            # Mean absolute percentage error
            MAPE = mean_absolute_percentage_error(y_test, model.predict(X_test))
            
            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'Accuracy:', 100-MAPE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), 100-MAPE]],
                                                                    columns=['TrialNumber', 'Parameters', 'Accuracy'] ))
    return(SearchResultsData)

In [45]:
# Calling the function
ResultsData =FunctionFindBestParams(X_train, y_train, X_test, y_test)

574/574 [==============================] - 2s 3ms/step
1 Parameters: batch_size: 10 - epochs: 10 Accuracy: 99.167245085856


/var/folders/0v/f9chsbzs06z9v0zl61n4lws40000gn/T/ipykernel_99887/2130507986.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), 100-MAPE]],


ValueError: too many values to unpack (expected 2)

In [46]:
def functionBestModel(batch, epoch):
    model = Sequential()
    # Defining the first layer of the model
    model.add(Dense(units=10, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))

    # applying dropout to layer
    model.add(Dropout(0.1))

    # Defining the Second layer of the model
    model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))

    # Defining the third layer of the model
    model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))

    # applying dropout to layer
    model.add(Dropout(0.1))

    # The output neuron is a single fully connected node 
    # Since we will be predicting a single number
    model.add(Dense(2, kernel_initializer='normal'))

    # Compiling the model
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Fitting the ANN to the Training set
    model.fit(X_train, y_train ,batch_size = batch, epochs = epoch, verbose=2)

    return model

In [47]:
model = functionBestModel(10, 10)


Epoch 1/10
4284/4284 - 18s - loss: 0.9087 - 18s/epoch - 4ms/step
Epoch 2/10
4284/4284 - 14s - loss: 0.8600 - 14s/epoch - 3ms/step
Epoch 3/10
4284/4284 - 8s - loss: 0.8463 - 8s/epoch - 2ms/step
Epoch 4/10
4284/4284 - 8s - loss: 0.8410 - 8s/epoch - 2ms/step
Epoch 5/10
4284/4284 - 7s - loss: 0.8358 - 7s/epoch - 2ms/step
Epoch 6/10
4284/4284 - 7s - loss: 0.8342 - 7s/epoch - 2ms/step
Epoch 7/10
4284/4284 - 12s - loss: 0.8335 - 12s/epoch - 3ms/step
Epoch 8/10
4284/4284 - 7s - loss: 0.8310 - 7s/epoch - 2ms/step
Epoch 9/10
4284/4284 - 10s - loss: 0.8264 - 10s/epoch - 2ms/step
Epoch 10/10
4284/4284 - 7s - loss: 0.8256 - 7s/epoch - 2ms/step


In [48]:
# Generating Predictions on testing data
Predictions=model.predict(X_test)
 
# Scaling the predicted Price data back to original price scale
Predictions=TargetVarScalerFit.inverse_transform(Predictions)
 
# Scaling the y_test Price data back to original price scale
y_test_orig=TargetVarScalerFit.inverse_transform(y_test)
 
# Scaling the test data back to original scale
Test_Data=PredictorScalerFit.inverse_transform(X_test)

574/574 [==============================] - 4s 6ms/step


In [49]:
TestingData=pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData[['Valence', 'Arousal']]=y_test_orig
TestingData[['PredictedValence', 'PredictedArousal']]=Predictions
TestingData.head()

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,Valence,Arousal,PredictedValence,PredictedArousal
0,-13.6685,2.4037,0.51566,6.3653,6.1908,-9.4808,6.13390,6.9684,-1.0656,-2.8258,-3.6370,-8.2337,5.3518,-19.2642,1.0,1.0,0.560713,0.764184
1,-8.3297,4.8245,-6.48240,7.6264,-11.6985,8.3160,-3.95040,-19.4175,4.0688,-17.3575,-10.7790,-3.7511,9.2850,6.7191,1.0,1.0,0.488513,0.321006
2,12.8005,-1.3156,0.70434,-20.0760,-16.7250,-13.5455,-17.95490,7.2087,-6.1483,14.5709,9.0522,10.1700,-12.5708,-1.8002,0.0,1.0,0.558705,0.777328
3,-4.4190,-20.6000,-21.78960,-13.2994,-22.0840,-9.0131,-10.18450,13.5013,-1.3261,8.2239,8.0685,5.4493,3.3099,-17.2251,0.0,0.0,0.386243,0.170813
4,-7.2066,2.6147,-4.74540,13.1577,1.6130,4.1389,-0.70903,-8.3539,-18.2423,-19.1286,-4.2966,-5.5636,-13.9850,-5.3433,1.0,1.0,0.582263,0.494960
